In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.layers import Dense, LSTM, GRU, Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import numpy as np
import optuna

In [51]:
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
min_max_scaler = MinMaxScaler()
window_size = 5
horizon = 1

# Functions

In [91]:
#
def currentdata(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)
    data.reset_index(inplace=True)
    data['Date'] = data['Date'].dt.strftime('%d-%m-%Y')
    
    def str_to_datetime(s):
        split = s.split('-')
        day, month, year = int(split[0]), int(split[1]), int(split[2])
        return datetime.datetime(day=day, month=month, year=year)
    data['Date'] = data['Date'].apply(str_to_datetime)
    
    first_row = data.iloc[:1]
    rest_of_data = data.iloc[1:]
    rest_of_data = rest_of_data[rest_of_data['Volume'] != 0]

    data = pd.concat([first_row, rest_of_data])
    data = data[['Date', 'Close']]
    data.index = data.pop('Date')
    return data

#
def create_dataset(data, window_size, horizon):
    X, y = [], []
    for i in range(len(data) - window_size - horizon + 1):
        X.append(data[i:(i + window_size)])
        y.append(data[(i + window_size):(i + window_size + horizon)])
    return np.array(X), np.array(y)

#
def create_model(model_type, window_size, horizon, units):
    model = Sequential()
    
    # Add the recurrent layer
    if model_type == 'LSTM':
        model.add(LSTM(units, activation = 'relu', return_sequences=False, input_shape=(window_size,1)))
    elif model_type == 'GRU':
        model.add(GRU(units, activation = 'relu', return_sequences=False, input_shape=(window_size,1)))
    
    # Add the output layer
    model.add(Dense(horizon))
    
    # Compile the model
    model.compile(optimizer='adam', loss='mae')
    
    return model

def create_windows_horizons(series, window_size, horizon_size):
    X, y = [], []
    for i in range(len(series) - window_size - horizon_size + 1):
        X.append(series[i:(i + window_size)])
        y.append(series[i + window_size:i + window_size + horizon_size])
    return np.array(X), np.array(y).reshape(-1, horizon_size)

def create_model_lstm(trial):
    model = Sequential()
    
    # Defining the number of units in LSTM layers
    lstm_units = trial.suggest_categorical('lstm_units', [32, 64, 128])
    
    # Defining dropout rate
    # dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5, step=0.1)
    
    model.add(LSTM(units=lstm_units, return_sequences=False, input_shape=(5, 1)))
    # model.add(Dropout(dropout_rate))
    
    model.add(Dense(units=1))  # Output layer
    model.compile(loss="mae", optimizer="adam")

    return model

def create_model_gru(trial):
    model = Sequential()
    
    # Defining the number of units in LSTM layers
    gru_units = trial.suggest_categorical('gru_units', [32, 64, 128])

    model.add(GRU(units=gru_units, return_sequences=False, input_shape=(5, 1)))
    
    model.add(Dense(units=1))  # Output layer
    model.compile(loss="mae", optimizer="adam")

    return model

def objective_lstm(trial):
    close_prices = train_df['Close'].values

    X, y = create_windows_horizons(close_prices, 5, 1)
    
    # Define the cross-validation technique
    tscv = TimeSeriesSplit(n_splits=5)
    
    val_losses = []  # List to store validation losses for each fold
    
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model = create_model_lstm(trial)
        early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
        history = model.fit(X_train, y_train,
                            validation_data=(X_test, y_test),
                            epochs=200,
                            batch_size=trial.suggest_categorical('batch_size', [32, 64, 128]),
                            callbacks=[early_stopping],
                            verbose=0)
        
        val_losses.append(min(history.history['val_loss']))
    
    # Calculate average validation loss across all folds
    avg_val_loss = np.mean(val_losses)
    return avg_val_loss

def objective_gru(trial):
    close_prices = train_df['Close'].values

    X, y = create_windows_horizons(close_prices, 5, 1)
    
    # Define the cross-validation technique
    tscv = TimeSeriesSplit(n_splits=5)
    
    val_losses = []  # List to store validation losses for each fold
    
    for train_index, test_index in tscv.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model = create_model_gru(trial)
        early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
        history = model.fit(X_train, y_train,
                            validation_data=(X_test, y_test),
                            epochs=200,
                            batch_size=trial.suggest_categorical('batch_size', [32, 64, 128]),
                            callbacks=[early_stopping],
                            verbose=0)
        
        val_losses.append(min(history.history['val_loss']))
    
    # Calculate average validation loss across all folds
    avg_val_loss = np.mean(val_losses)
    return avg_val_loss

# Optuna Hyperparameter Tuning
def optimize_lstm():
    study = optuna.create_study(direction='minimize')
    study.optimize(objective_lstm, n_trials=30)
    best_params = study.best_params
    print("Best Hyperparameters:", best_params)
def optimize_gru():
    study = optuna.create_study(direction='minimize')
    study.optimize(objective_gru, n_trials=30)
    best_params = study.best_params
    print("Best Hyperparameters:", best_params)

# Data Preparation

In [96]:
# set date for train-test dataset
start_train = datetime.datetime(2019, 1, 1)
end_train = start_test = datetime.datetime(2024, 1, 1)
end_test = datetime.datetime(2024, 5, 1)

In [118]:
data = yf.Ticker("BBCA.JK")
train_df = data.history(start=start_train, end=end_train)

# Buang data yang Volume = 0
# train_df = train_df[train_df['Volume'] != 0]

# train_df

In [101]:
test_df = data.history(start=start_test, end=end_test)
# test_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-01-02 00:00:00+07:00,9188.267221,9212.704102,9139.393459,9212.704102,30545200,0.0,0.0
2024-01-03 00:00:00+07:00,9212.704198,9212.704198,9114.956674,9139.393555,34603300,0.0,0.0
2024-01-04 00:00:00+07:00,9188.267480,9261.578125,9163.830599,9261.578125,56501800,0.0,0.0
2024-01-05 00:00:00+07:00,9286.014554,9383.762076,9261.577674,9359.325195,85290100,0.0,0.0
2024-01-08 00:00:00+07:00,9383.762076,9408.198956,9261.577674,9359.325195,50896000,0.0,0.0
...,...,...,...,...,...,...,...
2024-04-24 00:00:00+07:00,9825.000000,9950.000000,9750.000000,9950.000000,132832700,0.0,0.0
2024-04-25 00:00:00+07:00,10000.000000,10000.000000,9700.000000,9775.000000,104874100,0.0,0.0
2024-04-26 00:00:00+07:00,9700.000000,9825.000000,9625.000000,9625.000000,100932900,0.0,0.0


In [ ]:
data = currentdata("BBCA.JK", start_train, end_train)

plt.figure(figsize=(12,6))
plt.plot(data.index, data['Close'])
plt.xlabel('Date')
plt.ylabel('Closing Price IDR')
plt.show

# Cross Validation

In [92]:
data = currentdata("BBCA.JK", start_train, end_train)
data

[*********************100%%**********************]  1 of 1 completed


,Close
Date,
2019-01-01,5200.0
2019-01-02,5240.0
2019-01-03,5180.0
2019-01-04,5205.0
2019-01-07,5245.0
...,...
2023-12-21,9325.0
2023-12-22,9325.0
2023-12-27,9375.0


In [93]:
data = currentdata("BBCA.JK", start_train, end_train)
optimize_lstm()
optimize_gru()

# LSTM 128 32

[*********************100%%**********************]  1 of 1 completed
[I 2024-06-20 08:51:37,957] A new study created in memory with name: no-name-8f50ebe5-0583-4e4f-8714-0d1abc3594d1
[I 2024-06-20 08:53:35,076] Trial 0 finished with value: 3888.11552734375 and parameters: {'lstm_units': 128, 'batch_size': 64}. Best is trial 0 with value: 3888.11552734375.
[I 2024-06-20 08:55:29,711] Trial 1 finished with value: 3892.44189453125 and parameters: {'lstm_units': 128, 'batch_size': 64}. Best is trial 0 with value: 3888.11552734375.
[I 2024-06-20 08:56:44,642] Trial 2 finished with value: 3919.7828125 and parameters: {'lstm_units': 64, 'batch_size': 64}. Best is trial 0 with value: 3888.11552734375.
[I 2024-06-20 08:57:53,111] Trial 3 finished with value: 3937.04375 and parameters: {'lstm_units': 64, 'batch_size': 128}. Best is trial 0 with value: 3888.11552734375.
[I 2024-06-20 08:59:01,937] Trial 4 finished with value: 3933.3484375 and parameters: {'lstm_units': 32, 'batch_size': 64}. Best

Best Hyperparameters: {'lstm_units': 128, 'batch_size': 32}


[I 2024-06-20 09:46:07,815] Trial 0 finished with value: 3817.888232421875 and parameters: {'gru_units': 64, 'batch_size': 32}. Best is trial 0 with value: 3817.888232421875.
[I 2024-06-20 09:47:31,512] Trial 1 finished with value: 3886.8013671875 and parameters: {'gru_units': 128, 'batch_size': 128}. Best is trial 0 with value: 3817.888232421875.
[I 2024-06-20 09:49:36,668] Trial 2 finished with value: 3702.83388671875 and parameters: {'gru_units': 128, 'batch_size': 32}. Best is trial 2 with value: 3702.83388671875.
[I 2024-06-20 09:50:57,591] Trial 3 finished with value: 3825.673681640625 and parameters: {'gru_units': 64, 'batch_size': 32}. Best is trial 2 with value: 3702.83388671875.
[I 2024-06-20 09:51:56,606] Trial 4 finished with value: 3913.920361328125 and parameters: {'gru_units': 32, 'batch_size': 64}. Best is trial 2 with value: 3702.83388671875.
[I 2024-06-20 09:52:55,987] Trial 5 finished with value: 3916.06123046875 and parameters: {'gru_units': 64, 'batch_size': 128}. 

Best Hyperparameters: {'gru_units': 128, 'batch_size': 32}


In [94]:
data = currentdata("BBRI.JK", start_train, end_train)
optimize_lstm()
optimize_gru()

[*********************100%%**********************]  1 of 1 completed
[I 2024-06-20 10:34:00,715] A new study created in memory with name: no-name-2d298995-09ef-458d-8d31-72b797463416
[I 2024-06-20 10:35:27,204] Trial 0 finished with value: 3897.22529296875 and parameters: {'lstm_units': 64, 'batch_size': 32}. Best is trial 0 with value: 3897.22529296875.
[I 2024-06-20 10:37:29,811] Trial 1 finished with value: 3840.215625 and parameters: {'lstm_units': 128, 'batch_size': 32}. Best is trial 1 with value: 3840.215625.
[I 2024-06-20 10:38:44,429] Trial 2 finished with value: 3922.359033203125 and parameters: {'lstm_units': 64, 'batch_size': 64}. Best is trial 1 with value: 3840.215625.
[I 2024-06-20 10:39:34,914] Trial 3 finished with value: 3943.807177734375 and parameters: {'lstm_units': 32, 'batch_size': 128}. Best is trial 1 with value: 3840.215625.
[I 2024-06-20 10:40:53,377] Trial 4 finished with value: 3918.8677734375 and parameters: {'lstm_units': 32, 'batch_size': 32}. Best is tr

Best Hyperparameters: {'lstm_units': 128, 'batch_size': 32}


[I 2024-06-20 11:25:47,627] Trial 0 finished with value: 3919.66337890625 and parameters: {'gru_units': 32, 'batch_size': 64}. Best is trial 0 with value: 3919.66337890625.
[I 2024-06-20 11:27:08,422] Trial 1 finished with value: 3825.4994140625 and parameters: {'gru_units': 64, 'batch_size': 32}. Best is trial 1 with value: 3825.4994140625.
[I 2024-06-20 11:27:58,748] Trial 2 finished with value: 3933.068310546875 and parameters: {'gru_units': 32, 'batch_size': 128}. Best is trial 1 with value: 3825.4994140625.
[I 2024-06-20 11:29:18,484] Trial 3 finished with value: 3878.985400390625 and parameters: {'gru_units': 32, 'batch_size': 32}. Best is trial 1 with value: 3825.4994140625.
[I 2024-06-20 11:30:37,206] Trial 4 finished with value: 3822.29267578125 and parameters: {'gru_units': 64, 'batch_size': 32}. Best is trial 4 with value: 3822.29267578125.
[I 2024-06-20 11:31:40,264] Trial 5 finished with value: 3889.7935546875 and parameters: {'gru_units': 64, 'batch_size': 64}. Best is tr

Best Hyperparameters: {'gru_units': 128, 'batch_size': 32}


In [95]:
data = currentdata("BMRI.JK", start_train, end_train)
optimize_lstm()
optimize_gru()

[*********************100%%**********************]  1 of 1 completed
[I 2024-06-20 12:12:01,224] A new study created in memory with name: no-name-214aa4f9-aa0f-4da6-b691-7fefdc1c604c
[I 2024-06-20 12:12:51,133] Trial 0 finished with value: 3944.537451171875 and parameters: {'lstm_units': 32, 'batch_size': 128}. Best is trial 0 with value: 3944.537451171875.
[I 2024-06-20 12:13:41,519] Trial 1 finished with value: 3944.838427734375 and parameters: {'lstm_units': 32, 'batch_size': 128}. Best is trial 0 with value: 3944.537451171875.
[I 2024-06-20 12:14:41,255] Trial 2 finished with value: 3937.9712890625 and parameters: {'lstm_units': 64, 'batch_size': 128}. Best is trial 2 with value: 3937.9712890625.
[I 2024-06-20 12:15:53,329] Trial 3 finished with value: 3923.82744140625 and parameters: {'lstm_units': 64, 'batch_size': 64}. Best is trial 3 with value: 3923.82744140625.
[I 2024-06-20 12:17:51,223] Trial 4 finished with value: 3826.960498046875 and parameters: {'lstm_units': 128, 'batc

Best Hyperparameters: {'lstm_units': 128, 'batch_size': 32}


[I 2024-06-20 13:02:09,350] Trial 0 finished with value: 3918.24736328125 and parameters: {'gru_units': 64, 'batch_size': 128}. Best is trial 0 with value: 3918.24736328125.
[I 2024-06-20 13:03:08,328] Trial 1 finished with value: 3917.998974609375 and parameters: {'gru_units': 64, 'batch_size': 128}. Best is trial 1 with value: 3917.998974609375.
[I 2024-06-20 13:04:26,527] Trial 2 finished with value: 3884.107275390625 and parameters: {'gru_units': 128, 'batch_size': 128}. Best is trial 2 with value: 3884.107275390625.
[I 2024-06-20 13:05:41,976] Trial 3 finished with value: 3884.7119140625 and parameters: {'gru_units': 128, 'batch_size': 128}. Best is trial 2 with value: 3884.107275390625.
[I 2024-06-20 13:07:42,414] Trial 4 finished with value: 3692.602978515625 and parameters: {'gru_units': 128, 'batch_size': 32}. Best is trial 4 with value: 3692.602978515625.
[I 2024-06-20 13:08:41,296] Trial 5 finished with value: 3914.40322265625 and parameters: {'gru_units': 32, 'batch_size': 

Best Hyperparameters: {'gru_units': 128, 'batch_size': 32}


# Modeling

## BCA

### BCA Data Prep

In [102]:
# set date for train-test dataset
start_train = datetime.datetime(2019, 1, 1)
end_train = start_test = datetime.datetime(2024, 1, 1)
end_test = datetime.datetime(2024, 6, 1)
# end_test = datetime.datetime.now()


In [103]:
train_df = currentdata("BBCA.JK", start_train, end_train)

# Buang data yang Volume = 0
# train_df = train_df[train_df['Volume'] != 0]

# train_df

[*********************100%%**********************]  1 of 1 completed


In [104]:
test_df = currentdata("BBCA.JK", start_test, end_test)
# test_df

[*********************100%%**********************]  1 of 1 completed


### BCA LSTM

In [105]:
import os

best_model_mape = float('inf')
best_model_name = ""

close_train = train_df['Close'].values
close_test = test_df['Close'].values
date_train = train_df.index
date_test = test_df.index

# Konfigurasi window size dan horizon
configs = [
    {'window_size': 5, 'horizon': horizon, 'units':128, 'batch_size':32},
    {'window_size': 20, 'horizon': horizon, 'units':128, 'batch_size':32},
    {'window_size': 60, 'horizon': horizon, 'units':128, 'batch_size':32}
]

results = []

for config in configs:
    window_size = config['window_size']
    horizon = config['horizon']
    units = config['units']
    batch_size = config['batch_size']

    # Siapkan dataset
    X_train, y_train = create_windows_horizons(close_train, window_size, horizon)
    X_test, y_test = create_windows_horizons(close_test, window_size, horizon)

    # Buat dan latih model
    model = create_model('LSTM', window_size, horizon, units)
    early_stopping = EarlyStopping(monitor='loss', patience=150, restore_best_weights=True)
    lstmBCA = model.fit(
        X_train, y_train,
        # validation_data=(X_test, y_test),
        epochs=1000,
        batch_size = batch_size,
        callbacks=[early_stopping],
        verbose=0
        )
    
    # Prediksi pada data testing
    y_pred = model.predict(X_test)
    
    # Evaluasi model dengan MAE
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    model_name = f"BBCA_LSTM_window_{config['window_size']}.h5"
    model.save(model_name)
    
    # Update best model tracking
    if mape < best_model_mape:
        best_model_mape = mape
        if best_model_name:
            # Delete previous best model
            os.remove(best_model_name)
        best_model_name = model_name
    else:
        # Delete the current model as it's not the best
        os.remove(model_name)

    results.append({
        'config': config,
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'y_pred': y_pred,
        'y_test': y_test,
        'X_test': X_test
    })

# Tampilkan hasil
for result in results:
    config = result['config']
    mae = result['mae']
    rmse = result['rmse']
    mape = result['mape']
    y_pred = result['y_pred']
    y_test = result['y_test']
    X_test = result['X_test']
    
    print(f"Config: {config} - MAE: {mae} - RMSE: {rmse} - MAPE: {mape}")
    
# Config: {'window_size': 5, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 104.26784752155173 - RMSE: 138.97914244994882 - MAPE: 1.075615148223871
# Config: {'window_size': 20, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 120.23390028211806 - RMSE: 154.35554170431553 - MAPE: 1.2357405320062351
# Config: {'window_size': 60, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 758.2881164550781 - RMSE: 837.3033836771568 - MAPE: 8.007672405831027

1/1 [==============================] - 0s 154ms/step
Config: {'window_size': 5, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 104.26784752155173 - RMSE: 138.97914244994882 - MAPE: 1.075615148223871
Config: {'window_size': 20, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 120.23390028211806 - RMSE: 154.35554170431553 - MAPE: 1.2357405320062351
Config: {'window_size': 60, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 758.2881164550781 - RMSE: 837.3033836771568 - MAPE: 8.007672405831027


### BCA GRU

In [106]:
import os

best_model_mape = float('inf')
best_model_name = ""

close_train = train_df['Close'].values
close_test = test_df['Close'].values
date_train = train_df.index
date_test = test_df.index

# Konfigurasi window size dan horizon
configs = [
    {'window_size': 5, 'horizon': horizon, 'units':128, 'batch_size':32},
    {'window_size': 20, 'horizon': horizon, 'units':128, 'batch_size':32},
    {'window_size': 60, 'horizon': horizon, 'units':128, 'batch_size':32}
]

results = []

for config in configs:
    window_size = config['window_size']
    horizon = config['horizon']
    units = config['units']
    batch_size = config['batch_size']

    # Siapkan dataset
    X_train, y_train = create_windows_horizons(close_train, window_size, horizon)
    X_test, y_test = create_windows_horizons(close_test, window_size, horizon)

    # Buat dan latih model
    model = create_model('GRU', window_size, horizon, units)
    early_stopping = EarlyStopping(monitor='loss', patience=150, restore_best_weights=True)
    lstmBCA = model.fit(
        X_train, y_train,
        # validation_data=(X_test, y_test),
        epochs=1000,
        batch_size = batch_size,
        callbacks=[early_stopping],
        verbose=0
        )
    
    # Prediksi pada data testing
    y_pred = model.predict(X_test)
    
    # Evaluasi model dengan MAE
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    model_name = f"BBCA_GRU_window_{config['window_size']}.h5"
    model.save(model_name)
    
    # Update best model tracking
    if mape < best_model_mape:
        best_model_mape = mape
        if best_model_name:
            # Delete previous best model
            os.remove(best_model_name)
        best_model_name = model_name
    else:
        # Delete the current model as it's not the best
        os.remove(model_name)

    results.append({
        'config': config,
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'y_pred': y_pred,
        'y_test': y_test,
        'X_test': X_test
    })

# Tampilkan hasil
for result in results:
    config = result['config']
    mae = result['mae']
    rmse = result['rmse']
    mape = result['mape']
    y_pred = result['y_pred']
    y_test = result['y_test']
    X_test = result['X_test']
    
    print(f"Config: {config} - MAE: {mae} - RMSE: {rmse} - MAPE: {mape}")
    
# Config: {'window_size': 5, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 106.87752559267241 - RMSE: 142.2410524625235 - MAPE: 1.1011257528217848
# Config: {'window_size': 20, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 127.42656792534723 - RMSE: 160.23918828208994 - MAPE: 1.3155272112605274
# Config: {'window_size': 60, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 151.43734741210938 - RMSE: 189.3256661688393 - MAPE: 1.5848971669555938

1/1 [==============================] - 0s 164ms/step
Config: {'window_size': 5, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 106.87752559267241 - RMSE: 142.2410524625235 - MAPE: 1.1011257528217848
Config: {'window_size': 20, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 127.42656792534723 - RMSE: 160.23918828208994 - MAPE: 1.3155272112605274
Config: {'window_size': 60, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 151.43734741210938 - RMSE: 189.3256661688393 - MAPE: 1.5848971669555938


## BRI

### BRI Data Prep

In [107]:
# set date for train-test dataset
start_train = datetime.datetime(2019, 1, 1)
end_train = start_test = datetime.datetime(2024, 1, 1)
end_test = datetime.datetime(2024, 6, 1)

In [108]:
train_df = currentdata("BBRI.JK", start_train, end_train)
# train_df

[*********************100%%**********************]  1 of 1 completed


In [109]:
test_df = currentdata("BBRI.JK", start_test, end_test)
# test_df

[*********************100%%**********************]  1 of 1 completed


### BRI LSTM

In [110]:
import os

best_model_mape = float('inf')
best_model_name = ""

close_train = train_df['Close'].values
close_test = test_df['Close'].values
date_train = train_df.index
date_test = test_df.index

# Konfigurasi window size dan horizon
configs = [
    {'window_size': 5, 'horizon': horizon, 'units':128, 'batch_size':32},
    {'window_size': 20, 'horizon': horizon, 'units':128, 'batch_size':32},
    {'window_size': 60, 'horizon': horizon, 'units':128, 'batch_size':32}
]

results = []

for config in configs:
    window_size = config['window_size']
    horizon = config['horizon']
    units = config['units']
    batch_size = config['batch_size']

    # Siapkan dataset
    X_train, y_train = create_windows_horizons(close_train, window_size, horizon)
    X_test, y_test = create_windows_horizons(close_test, window_size, horizon)

    # Buat dan latih model
    model = create_model('LSTM', window_size, horizon, units)
    early_stopping = EarlyStopping(monitor='loss', patience=150, restore_best_weights=True)
    lstmBCA = model.fit(
        X_train, y_train,
        # validation_data=(X_test, y_test),
        epochs=1000,
        batch_size = batch_size,
        callbacks=[early_stopping],
        verbose=0
        )
    
    # Prediksi pada data testing
    y_pred = model.predict(X_test)
    
    # Evaluasi model dengan MAE
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    model_name = f"BBRI_LSTM_window_{config['window_size']}.h5"
    model.save(model_name)
    
    # Update best model tracking
    if mape < best_model_mape:
        best_model_mape = mape
        if best_model_name:
            # Delete previous best model
            os.remove(best_model_name)
        best_model_name = model_name
    else:
        # Delete the current model as it's not the best
        os.remove(model_name)

    results.append({
        'config': config,
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'y_pred': y_pred,
        'y_test': y_test,
        'X_test': X_test
    })

# Tampilkan hasil
for result in results:
    config = result['config']
    mae = result['mae']
    rmse = result['rmse']
    mape = result['mape']
    y_pred = result['y_pred']
    y_test = result['y_test']
    X_test = result['X_test']
    
    print(f"Config: {config} - MAE: {mae} - RMSE: {rmse} - MAPE: {mape}")
    
# Config: {'window_size': 5, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 84.78845074533046 - RMSE: 106.9815845185104 - MAPE: 1.5298559775936917
# Config: {'window_size': 20, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 199.3375244140625 - RMSE: 236.18623042887722 - MAPE: 3.633051521609845
# Config: {'window_size': 60, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 873.7863616943359 - RMSE: 963.0003404190292 - MAPE: 18.35443543313459

1/1 [==============================] - 0s 197ms/step
Config: {'window_size': 5, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 84.78845074533046 - RMSE: 106.9815845185104 - MAPE: 1.5298559775936917
Config: {'window_size': 20, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 199.3375244140625 - RMSE: 236.18623042887722 - MAPE: 3.633051521609845
Config: {'window_size': 60, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 873.7863616943359 - RMSE: 963.0003404190292 - MAPE: 18.35443543313459


### BRI GRU

In [111]:
import os

best_model_mape = float('inf')
best_model_name = ""

close_train = train_df['Close'].values
close_test = test_df['Close'].values
date_train = train_df.index
date_test = test_df.index

# Konfigurasi window size dan horizon
configs = [
    {'window_size': 5, 'horizon': horizon, 'units':128, 'batch_size':32},
    {'window_size': 20, 'horizon': horizon, 'units':128, 'batch_size':32},
    {'window_size': 60, 'horizon': horizon, 'units':128, 'batch_size':32}
]

results = []

for config in configs:
    window_size = config['window_size']
    horizon = config['horizon']
    units = config['units']
    batch_size = config['batch_size']

    # Siapkan dataset
    X_train, y_train = create_windows_horizons(close_train, window_size, horizon)
    X_test, y_test = create_windows_horizons(close_test, window_size, horizon)

    # Buat dan latih model
    model = create_model('GRU', window_size, horizon, units)
    early_stopping = EarlyStopping(monitor='loss', patience=150, restore_best_weights=True)
    lstmBCA = model.fit(
        X_train, y_train,
        # validation_data=(X_test, y_test),
        epochs=1000,
        batch_size = batch_size,
        callbacks=[early_stopping],
        verbose=0
        )
    
    # Prediksi pada data testing
    y_pred = model.predict(X_test)
    
    # Evaluasi model dengan MAE
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    model_name = f"BBRI_GRU_window_{config['window_size']}.h5"
    model.save(model_name)
    
    # Update best model tracking
    if mape < best_model_mape:
        best_model_mape = mape
        if best_model_name:
            # Delete previous best model
            os.remove(best_model_name)
        best_model_name = model_name
    else:
        # Delete the current model as it's not the best
        os.remove(model_name)

    results.append({
        'config': config,
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'y_pred': y_pred,
        'y_test': y_test,
        'X_test': X_test
    })

# Tampilkan hasil
for result in results:
    config = result['config']
    mae = result['mae']
    rmse = result['rmse']
    mape = result['mape']
    y_pred = result['y_pred']
    y_test = result['y_test']
    X_test = result['X_test']
    
    print(f"Config: {config} - MAE: {mae} - RMSE: {rmse} - MAPE: {mape}")
    
# Config: {'window_size': 5, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 86.17375516343391 - RMSE: 112.66981254160027 - MAPE: 1.576187108999173
# Config: {'window_size': 20, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 89.47710503472223 - RMSE: 113.88413822867058 - MAPE: 1.6298873769838578
# Config: {'window_size': 60, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 107.74046325683594 - RMSE: 137.43912899480176 - MAPE: 2.187604523873796

1/1 [==============================] - 0s 141ms/step
Config: {'window_size': 5, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 86.17375516343391 - RMSE: 112.66981254160027 - MAPE: 1.576187108999173
Config: {'window_size': 20, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 89.47710503472223 - RMSE: 113.88413822867058 - MAPE: 1.6298873769838578
Config: {'window_size': 60, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 107.74046325683594 - RMSE: 137.43912899480176 - MAPE: 2.187604523873796


## MRI

### MRI Data Prep

In [112]:
# set date for train-test dataset
start_train = datetime.datetime(2019, 1, 1)
end_train = start_test = datetime.datetime(2024, 1, 1)
end_test = datetime.datetime(2024, 6, 1)

In [113]:
train_df = currentdata("BMRI.JK", start_train, end_train)
# train_df

[*********************100%%**********************]  1 of 1 completed


In [114]:
test_df = currentdata("BMRI.JK", start_test, end_test)
# test_df

[*********************100%%**********************]  1 of 1 completed


### MRI LSTM

In [115]:
import os

best_model_mape = float('inf')
best_model_name = ""

close_train = train_df['Close'].values
close_test = test_df['Close'].values
date_train = train_df.index
date_test = test_df.index

# Konfigurasi window size dan horizon
configs = [
    {'window_size': 5, 'horizon': horizon, 'units':128, 'batch_size':32},
    {'window_size': 20, 'horizon': horizon, 'units':128, 'batch_size':32},
    {'window_size': 60, 'horizon': horizon, 'units':128, 'batch_size':32}
]

results = []

for config in configs:
    window_size = config['window_size']
    horizon = config['horizon']
    units = config['units']
    batch_size = config['batch_size']

    # Siapkan dataset
    X_train, y_train = create_windows_horizons(close_train, window_size, horizon)
    X_test, y_test = create_windows_horizons(close_test, window_size, horizon)

    # Buat dan latih model
    model = create_model('LSTM', window_size, horizon, units)
    early_stopping = EarlyStopping(monitor='loss', patience=150, restore_best_weights=True)
    lstmBCA = model.fit(
        X_train, y_train,
        # validation_data=(X_test, y_test),
        epochs=1000,
        batch_size = batch_size,
        callbacks=[early_stopping],
        verbose=0
        )
    
    # Prediksi pada data testing
    y_pred = model.predict(X_test)
    
    # Evaluasi model dengan MAE
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    model_name = f"BMRI_LSTM_window_{config['window_size']}.h5"
    model.save(model_name)
    
    # Update best model tracking
    if mape < best_model_mape:
        best_model_mape = mape
        if best_model_name:
            # Delete previous best model
            os.remove(best_model_name)
        best_model_name = model_name
    else:
        # Delete the current model as it's not the best
        os.remove(model_name)

    results.append({
        'config': config,
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'y_pred': y_pred,
        'y_test': y_test,
        'X_test': X_test
    })

# Tampilkan hasil
for result in results:
    config = result['config']
    mae = result['mae']
    rmse = result['rmse']
    mape = result['mape']
    y_pred = result['y_pred']
    y_test = result['y_test']
    X_test = result['X_test']
    
    print(f"Config: {config} - MAE: {mae} - RMSE: {rmse} - MAPE: {mape}")
    
# Config: {'window_size': 5, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 103.19848352191092 - RMSE: 142.20988779551521 - MAPE: 1.5562594510536465
# Config: {'window_size': 20, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 105.76382785373264 - RMSE: 148.6879434855225 - MAPE: 1.5839837189561365
# Config: {'window_size': 60, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 538.0663757324219 - RMSE: 738.6805651547733 - MAPE: 8.780960237354076

1/1 [==============================] - 0s 188ms/step
Config: {'window_size': 5, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 103.19848352191092 - RMSE: 142.20988779551521 - MAPE: 1.5562594510536465
Config: {'window_size': 20, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 105.76382785373264 - RMSE: 148.6879434855225 - MAPE: 1.5839837189561365
Config: {'window_size': 60, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 538.0663757324219 - RMSE: 738.6805651547733 - MAPE: 8.780960237354076


### MRI GRU

In [116]:
import os

best_model_mape = float('inf')
best_model_name = ""

close_train = train_df['Close'].values
close_test = test_df['Close'].values
date_train = train_df.index
date_test = test_df.index

# Konfigurasi window size dan horizon
configs = [
    {'window_size': 5, 'horizon': horizon, 'units':128, 'batch_size':32},
    {'window_size': 20, 'horizon': horizon, 'units':128, 'batch_size':32},
    {'window_size': 60, 'horizon': horizon, 'units':128, 'batch_size':32}
]
results = []

for config in configs:
    window_size = config['window_size']
    horizon = config['horizon']
    units = config['units']
    batch_size = config['batch_size']

    # Siapkan dataset
    X_train, y_train = create_windows_horizons(close_train, window_size, horizon)
    X_test, y_test = create_windows_horizons(close_test, window_size, horizon)

    # Buat dan latih model
    model = create_model('GRU', window_size, horizon, units)
    early_stopping = EarlyStopping(monitor='loss', patience=150, restore_best_weights=True)
    lstmBCA = model.fit(
        X_train, y_train,
        # validation_data=(X_test, y_test),
        epochs=1000,
        batch_size = batch_size,
        callbacks=[early_stopping],
        verbose=0
        )
    
    # Prediksi pada data testing
    y_pred = model.predict(X_test)
    
    # Evaluasi model dengan MAE
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    model_name = f"BMRI_GRU_window_{config['window_size']}.h5"
    model.save(model_name)
    
    # Update best model tracking
    if mape < best_model_mape:
        best_model_mape = mape
        if best_model_name:
            # Delete previous best model
            os.remove(best_model_name)
        best_model_name = model_name
    else:
        # Delete the current model as it's not the best
        os.remove(model_name)

    results.append({
        'config': config,
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'y_pred': y_pred,
        'y_test': y_test,
        'X_test': X_test
    })

# Tampilkan hasil
for result in results:
    config = result['config']
    mae = result['mae']
    rmse = result['rmse']
    mape = result['mape']
    y_pred = result['y_pred']
    y_test = result['y_test']
    X_test = result['X_test']
    
    print(f"Config: {config} - MAE: {mae} - RMSE: {rmse} - MAPE: {mape}")
    
# Config: {'window_size': 5, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 95.17061781609195 - RMSE: 133.92838950989994 - MAPE: 1.429475067198436
# Config: {'window_size': 20, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 111.00375705295139 - RMSE: 153.13703546906822 - MAPE: 1.6667123215036383
# Config: {'window_size': 60, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 126.78852844238281 - RMSE: 167.10145373523466 - MAPE: 1.9783773296786649

1/1 [==============================] - 0s 160ms/step
Config: {'window_size': 5, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 95.17061781609195 - RMSE: 133.92838950989994 - MAPE: 1.429475067198436
Config: {'window_size': 20, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 111.00375705295139 - RMSE: 153.13703546906822 - MAPE: 1.6667123215036383
Config: {'window_size': 60, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 126.78852844238281 - RMSE: 167.10145373523466 - MAPE: 1.9783773296786649


# Coba-coba

### BCA LSTM

In [ ]:
# BCA LSTM
# data = currentdata("BBCA.JK", start_train, end_train)
import os

best_model_mape = float('inf')
best_model_name = ""

close_train = train_df['Close'].values
close_test = test_df['Close'].values
date_train = train_df.index
date_test = test_df.index

# Konfigurasi window size dan horizon
configs = [
    {'window_size': 5, 'horizon': horizon, 'units':128, 'batch_size':32},
    {'window_size': 20, 'horizon': horizon, 'units':128, 'batch_size':32},
    {'window_size': 60, 'horizon': horizon, 'units':128, 'batch_size':32}
]

results = []

for config in configs:
    window_size = config['window_size']
    horizon = config['horizon']
    units = config['units']
    batch_size = config['batch_size']

    # Siapkan dataset
    X_train, y_train = create_windows_horizons(close_train, window_size, horizon)
    X_test, y_test = create_windows_horizons(close_test, window_size, horizon)

    # if X.shape[0] == 0:
    #     print(f"Dataset is too small for given window size and horizon")
    # else:
    #     X = X.reshape((X.shape[0], X.shape[1], 1))
    
    # # Bagi dataset menjadi training dan testing
    # split = int(0.8 * len(X))
    # X_train, X_test = X[:split], X[split:]
    # y_train, y_test = y[:split], y[split:]

    # Buat dan latih model
    model = create_model('LSTM', window_size, horizon, units)
    early_stopping = EarlyStopping(monitor='loss', patience=150, restore_best_weights=True)
    lstmBCA = model.fit(
        X_train, y_train,
        # validation_data=(X_test, y_test),
        epochs=1000,
        batch_size = batch_size,
        callbacks=[early_stopping],
        verbose=0
        )
    
    # Prediksi pada data testing
    y_pred = model.predict(X_test)
    
    # Evaluasi model dengan MAE
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    
    # y_pred = min_max_scaler.inverse_transform(y_pred)
    # y_test = min_max_scaler.inverse_transform(y_test)

    model_name = f"BCA_LSTM_window_{config['window_size']}.h5"
    model.save(model_name)
    
    # Update best model tracking
    if mape < best_model_mape:
        best_model_mape = mape
        if best_model_name:
            # Delete previous best model
            os.remove(best_model_name)
        best_model_name = model_name
    else:
        # Delete the current model as it's not the best
        os.remove(model_name)

    results.append({
        'config': config,
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'y_pred': y_pred,
        'y_test': y_test,
        'X_test': X_test
    })

# Tampilkan hasil
for result in results:
    config = result['config']
    mae = result['mae']
    rmse = result['rmse']
    mape = result['mape']
    y_pred = result['y_pred']
    y_test = result['y_test']
    X_test = result['X_test']
    
    print(f"Config: {config} - MAE: {mae} - RMSE: {rmse} - MAPE: {mape}")
    


1/1 [==============================] - 0s 360ms/step
Config: {'window_size': 5, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 109.2073001585145 - RMSE: 139.47296116801786 - MAPE: 1.1334455474420757
Config: {'window_size': 20, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 109.28993055555556 - RMSE: 145.15036202104992 - MAPE: 1.1250623907907198
Config: {'window_size': 60, 'horizon': 1, 'units': 128, 'batch_size': 32} - MAE: 1299.995396205357 - RMSE: 1320.635031486181 - MAPE: 13.504779775797074


### BCA GRU

In [ ]:
# BCA LSTM
# data = currentdata("BBCA.JK", start_train, end_train)
import os

best_model_mape = float('inf')
best_model_name = ""

close_train = train_df['Close'].values
close_test = test_df['Close'].values
date_train = train_df.index
date_test = test_df.index

# Konfigurasi window size dan horizon
configs = [
    {'window_size': 5, 'horizon': horizon, 'units':32, 'batch_size':64},
    {'window_size': 20, 'horizon': horizon, 'units':32, 'batch_size':64},
    {'window_size': 60, 'horizon': horizon, 'units':32, 'batch_size':64}
]

results = []

for config in configs:
    window_size = config['window_size']
    horizon = config['horizon']
    units = config['units']
    batch_size = config['batch_size']

    # Siapkan dataset
    X_train, y_train = create_windows_horizons(close_train, window_size, horizon)
    X_test, y_test = create_windows_horizons(close_test, window_size, horizon)

    # if X.shape[0] == 0:
    #     print(f"Dataset is too small for given window size and horizon")
    # else:
    #     X = X.reshape((X.shape[0], X.shape[1], 1))
    
    # # Bagi dataset menjadi training dan testing
    # split = int(0.8 * len(X))
    # X_train, X_test = X[:split], X[split:]
    # y_train, y_test = y[:split], y[split:]

    # Buat dan latih model
    model = create_model('GRU', window_size, horizon, units)
    early_stopping = EarlyStopping(monitor='loss', patience=150, restore_best_weights=True)
    lstmBCA = model.fit(
        X_train, y_train,
        # validation_data=(X_test, y_test),
        epochs=1000,
        batch_size = batch_size,
        callbacks=[early_stopping],
        verbose=0
        )
    
    # Prediksi pada data testing
    y_pred = model.predict(X_test)
    
    # Evaluasi model dengan MAE
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    
    # y_pred = min_max_scaler.inverse_transform(y_pred)
    # y_test = min_max_scaler.inverse_transform(y_test)

    model_name = f"BCA_GRU_window_{config['window_size']}.h5"
    model.save(model_name)
    
    # Update best model tracking
    if mape < best_model_mape:
        best_model_mape = mape
        if best_model_name:
            # Delete previous best model
            os.remove(best_model_name)
        best_model_name = model_name
    else:
        # Delete the current model as it's not the best
        os.remove(model_name)

    results.append({
        'config': config,
        'mae': mae,
        'rmse': rmse,
        'mape': mape,
        'y_pred': y_pred,
        'y_test': y_test,
        'X_test': X_test
    })

# Tampilkan hasil
for result in results:
    config = result['config']
    mae = result['mae']
    rmse = result['rmse']
    mape = result['mape']
    y_pred = result['y_pred']
    y_test = result['y_test']
    X_test = result['X_test']
    
    print(f"Config: {config} - MAE: {mae} - RMSE: {rmse} - MAPE: {mape}")
    
# Config: {'window_size': 5, 'horizon': 1, 'units': 32} - MAE: 0.012053401651227428 - RMSE: 0.01660147070614022 - MAPE: 1.4408710378368317
# Config: {'window_size': 20, 'horizon': 5, 'units': 32} - MAE: 0.01799186265265679 - RMSE: 0.024009757751589412 - MAPE: 2.1561077631718466
# Config: {'window_size': 120, 'horizon': 20, 'units': 32} - MAE: 0.027314258890569636 - RMSE: 0.03533805271756846 - MAPE: 3.2741308978164256

# Config: {'window_size': 5, 'horizon': 1, 'units': 32} - MAE: 0.012385194896864883 - RMSE: 0.016859943757688577 - MAPE: 1.481477461209736
# Config: {'window_size': 20, 'horizon': 5, 'units': 32} - MAE: 0.0179636667184997 - RMSE: 0.0240953834886749 - MAPE: 2.1517509504159666
# Config: {'window_size': 120, 'horizon': 20, 'units': 32} - MAE: 0.028443788657083875 - RMSE: 0.03707448897257839 - MAPE: 3.4168648909723336

# Config: {'window_size': 5, 'horizon': 1, 'units': 32, 'batch_size': 64} - MAE: 104.21180083786231 - RMSE: 137.58450292774046 - MAPE: 1.080471809864377
# Config: {'window_size': 20, 'horizon': 1, 'units': 32, 'batch_size': 64} - MAE: 163.10516131365742 - RMSE: 215.93862739996987 - MAPE: 1.6812227460370504
# Config: {'window_size': 60, 'horizon': 1, 'units': 32, 'batch_size': 64} - MAE: 189.39369419642858 - RMSE: 239.81157670769514 - MAPE: 1.9827377347315238

7/7 [==============================] - 0s 7ms/step
Config: {'window_size': 5, 'horizon': 1, 'units': 32} - MAE: 69.58751349720528 - RMSE: 94.5302131606018 - MAPE: 0.8130435919007695
Config: {'window_size': 20, 'horizon': 1, 'units': 32} - MAE: 71.85928256815843 - RMSE: 96.0438859475937 - MAPE: 0.8395024939539579
Config: {'window_size': 120, 'horizon': 1, 'units': 32} - MAE: 131.61455559837444 - RMSE: 162.40503030271995 - MAPE: 1.518751873624365


In [12]:
# window_size = 5
# horizon = 1

# data = currentdata("BBCA.JK", start_date, end_date)
# close_prices = data['Close'].values
# dates = data['Date'].values

# # Siapkan dataset
# X, y = create_dataset(close_prices, window_size, horizon)
# if X.shape[0] == 0:
#     print(f"Dataset is too small for given window size and horizon")
# else:
#     X = X.reshape((X.shape[0], X.shape[1], 1))

# # Bagi dataset menjadi training dan testing
# split = int(0.8 * len(X))
# X_train, X_test = X[:split], X[split:]
# y_train, y_test = y[:split], y[split:]

# # Buat dan latih model
# model = create_lstm_bca(window_size, horizon)
# early_stopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)
# lstmBCA = model.fit(
#     X_train, y_train,
#     validation_data=(X_test, y_test),
#     epochs=1000,
#     batch_size = 64,
#     verbose=0,
#     callbacks=[early_stopping]
#     )

# # # Memuat data yang akan diprediksi
# # data_to_predict = currentdata("BBCA.JK", start_date="2024-01-01", end_date="2024-04-30")
# # prices_to_predict = data_to_predict['Close'].values

# predictions = model.predict(X_test)

# # Evaluasi model dengan MAE
# mae = mean_absolute_error(y_test, predictions)

# # Inverse transform prediksi dan y_test ke skala asli
# predictions = min_max_scaler.inverse_transform(predictions)
# y_test = min_max_scaler.inverse_transform(y_test)

# # # Predict future values from Jan 2024 to Apr 2024
# # future_dates = []
# # future_predictions = []
# # last_window = close_prices[-window_size:].reshape((1, window_size, 1))

# # for i in range(120):  # Predicting for approximately 4 months (30 days/month)
# #     pred = model.predict(last_window)
# #     future_predictions.append(pred[0][0])
# #     last_window = np.roll(last_window, -1)
# #     last_window[0, -1, 0] = pred
# #     future_date = (datetime.datetime.strptime(dates[-1], '%d-%m-%Y') + datetime.timedelta(days=i + 1)).strftime('%d-%m-%Y')
# #     future_dates.append(future_date)

# # # Inverse transform predictions
# # future_predictions = np.array(future_predictions).reshape(-1, 1)
# # future_predictions = min_max_scaler.inverse_transform(future_predictions)

In [13]:
# # Plot actual and predicted values
# plt.figure(figsize=(12, 6))
# actual_prices = min_max_scaler.inverse_transform(close_prices.reshape(-1, 1))
# plt.plot(dates, actual_prices, label='Actual')
# plt.plot(future_dates, future_predictions.flatten(), label='Predicted')
# plt.title(f"Predictions for Jan 2024 to Apr 2024")
# plt.xlabel('Date')
# plt.ylabel('Close Price')
# plt.legend()

# plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
# plt.grid(True)
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()
# plt.savefig('test1.png', bbox_inches='tight')

In [14]:
# # Plot data sebenarnya dan prediksi
# plt.figure(figsize=(12, 6))
# # Menentukan indeks untuk plotting
# test_dates = dates[-len(y_test):] if len(dates) >= len(y_test) else dates

# plt.plot(test_dates, y_test, label='Actual')
# plt.plot(test_dates, predictions, label='Predicted')
# plt.title(f"Predictions for Jan 2024 to Apr 2024 - MAE: {mae:.5f}")
# plt.xlabel('Date')
# plt.ylabel('Close Price')
# plt.legend()

# plt.gca().xaxis.set_major_locator(mdates.MonthLocator(interval=1))
# plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
# plt.grid(True)
# plt.xticks(rotation=45)
# plt.tight_layout()
# plt.show()

# mape = np.mean(np.abs((y_test - predictions) / y_test)) * 100
# print(f"MAE: {mae}")
# print(f"MAPE: {mape}")
# plt.savefig('test2.png', bbox_inches='tight')

In [15]:
# def visualize_pred_actual(timesteps, data, data_actual, start=0, end=None, model_name="LSTM", errors=None):
#     if len(timesteps) == 1 or len(data) == 1 or len(data_actual) == 1:
#         data_timesteps = timesteps
#         pred_prices = [data]
#         actual_prices = data_actual
#     else:
#         data_timesteps = timesteps[start:end]
#         pred_prices = data[start:end]
#         actual_prices = data_actual[start:end]

#     fig = go.Figure()
#     fig.add_trace(go.Scatter(x=data_timesteps, y=pred_prices, name="Prediction", marker=dict(color='blue'), mode="markers"))
#     fig.add_trace(go.Scatter(x=data_timesteps, y=actual_prices, name="Actual", marker=dict(color='orange')))